# Hack@UCF 100% Organic NCCDC Scoring Engine 
This is a scoring engine for CCDC scrimmage and competition environments. Originally the basis of this program was written in python, but I had a few issues that were solved by Jupyter

1. Graphics. Jupyter + Numpy or other libraries makes visualization a breeze as compared to before
2. The ability to chain scripts. Part of the scoring engine requires the user to set some variables dependent on their environment. Given that Icinga doesn't flush it's databases each time you have an event, we can use this notebook to easily compile the information required per event


In [1]:
import pymysql.cursors
import configparser

## Step 1: Environment Configuration
Given that your environment is already setup within Icinga2, we must gather certain identifiers which Icinga uses internally in order to get the correct information from the tooling.

In this next step, we will define a few parameters pertaining to Icinga2's setup. Primarily pertaining to databases.

In [9]:
# Load in the credentials from the ini file
credentials = configparser.ConfigParser()
credentials.read('credentials.ini')

# Connect to the mysql server running on the icinga2 server
connection = pymysql.connect(host=credentials['database']['host'], 
                             user=credentials['database']['username'], 
                             password=credentials['database']['password'], 
                             db=credentials['database']['db'], 
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

## Step 2: Environment Enumeration and Host Selection
Now that we are able to connect to Icinga's internal database, we must select which hosts to generate scores for. Icinga doesn't flush the database holding all hosts per each event, so each time a new competition is held, the hosts selected will change. In order to ensure that the correct hosts are watched, we will compile a list of hostIDs.

In [77]:
import pandas as pd

query = "select host_object_id, display_name, address from icinga_hosts;"

cursor.execute(query)

results = cursor.fetchall()

df = pd.DataFrame.from_dict(results)

df.style.set_properties(**{'text-align': 'left'})

,address,display_name,host_object_id
0,127.0.0.1,icinga,219
1,192.168.50.104,AMediawiki,247
2,192.168.50.104,A Mediawiki,249
3,192.168.50.101,A Linux Wordpress,251
4,192.168.50.104,A Windows MediaWiki,252
5,192.168.50.102,A Linux Bind,253
6,192.168.50.106,A Windows Mail,254
7,192.168.50.105,A Windows AD/DNS,255
8,192.168.50.103,A Linux DB,256
9,192.168.50.102,A - Linux - Bind,263


Once you know what object IDs the hosts you are scoring will use, enter them into the following list:

In [28]:
scored_hosts = [332,331,330,329,328,327,326,325,324,323]

## Service Verification
This step will now complete a query on the selected hosts to see what services will be checked for each host. Please ensure that this is correct before moving on to the next step.

In [41]:
for host in scored_hosts:
    query = "select service_id, display_name from icinga_services where host_object_id = %s;" % host
    
    cursor.execute(query)
    
    services = cursor.fetchall()
    
    query = "select display_name from icinga_hosts where host_object_id = %d" % host
    
    cursor.execute(query)
    
    hostname = cursor.fetchone()['display_name']
    
    print("Host: [%s] %s" % (host,hostname))
    display(pd.DataFrame.from_dict(services))
    print("------------------------")

Host: [323] A - Windows - SMB/FTP


,display_name,service_id
0,smb,71
1,ftp,84
2,ping4,91


------------------------
Host: [324] A - Linux - Ghost (BLOG)


,display_name,service_id
0,http,68
1,ping4,69
2,ssh,80


------------------------
Host: [325] B - Windows - SMB/FTP


...

------------------------
Host: [331] A - Linux - MySQL/Bind


,display_name,service_id
0,dns,67
1,mysql,76
2,ping4,92


------------------------
Host: [332] A - Linux - Mail


,display_name,service_id
0,http,65
1,ping4,79
2,smtp,94


------------------------


## Step 3. Time Configuration
At this point, we must now determine what time range is to be scored. The values entered into the next section will determine the time range to be scored:

In [79]:
import time, datetime
time.strftime('%Y-%m-%d %H:%M:%S')

#datetime.datetime(year, month, day, hour, minute, {second}), Omittable starting from right (seconds usually omitted)
start_datetime = datetime.datetime(2018, 11, 1, 0, 0)
end_datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

## Step 4. SCORING! YAY!
Now we get to actual scoring. Each service's uptime will be calculated during the selected time range. Each service's uptime will be shown. Under each host will be it's individual uptime percentage.

In [78]:
from IPython.core.display import display, HTML
for host in scored_hosts:
    query = "select service_object_id, display_name from icinga_services where host_object_id = %s;" % host
    
    cursor.execute(query)
    
    services = cursor.fetchall()
    
    query = "select display_name from icinga_hosts where host_object_id = %d" % host
    
    cursor.execute(query)
    
    hostname = cursor.fetchone()['display_name']
    
    display(HTML("<b>Host: [%s] %s</b>" % (host,hostname)))
    
    uptimes = []
    uptime_sum = 0;
    for service in services:
        query = "select icinga2.icinga_availability(%s,\"%s\",\"%s\") as uptime;" % (service['service_object_id'],start_datetime,end_datetime)

        cursor.execute(query)
        
        uptime = cursor.fetchone()['uptime']
        
        if(uptime != None):
            uptime_sum = uptime_sum + float(uptime)
        
        uptimes.append({"name":service['display_name'], "uptime": uptime})
        
    display(pd.DataFrame.from_dict(uptimes))
    
    
    display(HTML("<i>Average Uptime: %s</i>" % (uptime_sum/len(uptimes))))
    display(HTML("<===========================>"))
        

,name,uptime
0,smb,None
1,ftp,9.2040
2,ping4,9.2055


,name,uptime
0,http,None
1,ping4,None
2,ssh,None


,name,uptime
0,ping4,None
1,smb,None
2,ftp,None


,name,uptime
0,dns,None
1,ping4,None
2,mysql,None


,name,uptime
0,ping4,9.2055
1,http,None
2,smtp,None


,name,uptime
0,mysql,None
1,http,None
2,ping4,None


,name,uptime
0,ping4,9.2018
1,http,9.2070
2,mysql,None


,name,uptime
0,http,None
1,ssh,None
2,ping4,None


,name,uptime
0,dns,None
1,mysql,None
2,ping4,None


,name,uptime
0,http,0.1055
1,ping4,9.2002
2,smtp,9.1504
